# 1 perpare minikube

## load environment variables

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/roboticbase-core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/roboticbase-core
```

In [ ]:
source ${PJ_ROOT}/docs/minikube/env

## start minikube

**select `A. without monitoring & logging` or `B. with monitoring & logging`**

### A. without monitoring & logging

In [ ]:
yes | minikube start --cpus 2 --memory 4096 --kubernetes-version v1.10.0

### B. with monitoring & logging

In [ ]:
yes | minikube start --cpus 4 --memory 8192 --kubernetes-version v1.10.0 --bootstrapper=kubeadm --extra-config=kubelet.authentication-token-webhook=true --extra-config=kubelet.authorization-mode=Webhook --extra-config=scheduler.address=0.0.0.0 --extra-config=controller-manager.address=0.0.0.0

expected)
```
Starting local Kubernetes v1.10.0 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.
```

## check network configuration of minikube
the following scripts are for macOS.

In [ ]:
NWNAME=$(VBoxManage showvminfo minikube | grep "Host-only Interface" | awk 'match($0, /vboxnet[0-9]+/){print substr($0,RSTART,RLENGTH)}');echo ${NWNAME}

example)
```
vboxnet0
```

In [ ]:
export HOST_IPADDR=$(ifconfig ${NWNAME} | awk '/inet / {print $2}')
sed -i '' -e "s/<<HOST_IPADDR>>/${HOST_IPADDR}/" ${PJ_ROOT}/docs/minikube/env
echo ${HOST_IPADDR}

example)
```
192.168.99.1
```

In [ ]:
NETMASK_HEX=$(ifconfig ${NWNAME} | awk '/netmask / {print $4}')
export NETMASK=$(echo "${NETMASK_HEX:2}" | perl -pe '$_ = unpack("B32", pack("H*", $_)); s/0+$//g; $_ = length')
echo ${NETMASK}

example)
```
24
```

## edit configuration of minikube

Edit `InsecureRegistry` of minikube in order to access local registory.

In [ ]:
if [ -f ${HOME}/.minikube/machines/minikube/config.json.org ]; then cp ${HOME}/.minikube/machines/minikube/config.json.org ${HOME}/.minikube/machines/minikube/config.json; else cp ${HOME}/.minikube/machines/minikube/config.json ${HOME}/.minikube/machines/minikube/config.json.org; fi
cat ${HOME}/.minikube/machines/minikube/config.json | perl -pse 's/"InsecureRegistry": \[/"InsecureRegistry": [\n                "$h\/$m",/g;' -- -h=${HOST_IPADDR} -m=${NETMASK} > /tmp/config.json
mv /tmp/config.json ${HOME}/.minikube/machines/minikube/config.json
diff -u ${HOME}/.minikube/machines/minikube/config.json.org ${HOME}/.minikube/machines/minikube/config.json

example)
```diff
--- /Users/nmatsui/.minikube/machines/minikube/config.json.org	2018-07-23 11:26:27.000000000 +0900
+++ /Users/nmatsui/.minikube/machines/minikube/config.json	2018-07-23 11:35:25.000000000 +0900
@@ -41,6 +41,7 @@
             "Env": null,
             "Ipv6": false,
             "InsecureRegistry": [
+                "192.168.99.1/24",
                 "10.96.0.0/12"
             ],
             "Labels": null,
             ```

## restart minikube

**select `A. without monitoring & logging` or `B. with monitoring & logging`**

### A. without monitoring & logging

In [ ]:
minikube stop
minikube start --cpus 2 --memory 4096 --kubernetes-version v1.10.0

### B. with monitoring & logging

In [ ]:
minikube stop
minikube start --cpus 4 --memory 8192 --kubernetes-version v1.10.0 --bootstrapper=kubeadm --extra-config=kubelet.authentication-token-webhook=true --extra-config=kubelet.authorization-mode=Webhook --extra-config=scheduler.address=0.0.0.0 --extra-config=controller-manager.address=0.0.0.0

example)
```
Stopping local Kubernetes cluster...
Machine stopped.
Starting local Kubernetes v1.10.0 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.
```

### confirm nodes

In [ ]:
kubectl get nodes

example)
```
NAME       STATUS    ROLES     AGE       VERSION
minikube   Ready     master    14m       v1.10.0
```

## start local registry on hostOS

Edit `InsecureRegistry` of local docker daemon in order to accept minikube.

**Docker for mac > Preference > Daemon > Advanced**

```diff
diff -u /tmp/docker-mac-configuration.json.org /tmp/docker-mac-configuration.json
--- /tmp/docker-mac-configuration.json.org	2018-05-15 09:41:37.000000000 +0900
+++ /tmp/docker-mac-configuration.json	2018-05-15 09:42:39.000000000 +0900
@@ -1,4 +1,7 @@
 {
   "debug" : true,
-  "experimental" : true
+  "experimental" : true,
+  "insecure-registries" : [
+    "192.168.99.0/24"
+  ]
 }
```

### start local registry

In [ ]:
docker run --name registry -p 5000:5000 -d registry:2.6

In [ ]:
export REPOSITORY=${HOST_IPADDR}:5000
sed -i '' -e "s/<<REPOSITORY>>/${REPOSITORY}/" ${PJ_ROOT}/docs/minikube/env
echo ${REPOSITORY}

### setup rbac

In [ ]:
kubectl apply -f rbac/tiller-rbac.yaml

In [ ]:
kubectl get serviceaccounts -n kube-system | grep tiller

expected)
```
tiller                               1         1m
```

In [ ]:
kubectl apply -f rbac/default-rbac.yaml

In [ ]:
kubectl get clusterroles | grep default-read

expected)
```
default-read                                                           1m
```

In [ ]:
kubectl get clusterrolebindings | grep default

expected)
```
default                                                2m
```

## setup helm

In [ ]:
helm version --client

expected)
```
Client: &version.Version{SemVer:"v2.10.0", GitCommit:"9ad53aac42165a5fadc6c87be0dea6b115f93090", GitTreeState:"clean"}
```

In [ ]:
helm init --service-account tiller

In [ ]:
helm repo update

In [ ]:
kubectl get pods --all-namespaces | grep tiller

expected)
```
kube-system   tiller-deploy-7ccf99cd64-gkn7t          1/1       Running   0          1m
```

In [ ]:
helm version

expected)
```
Client: &version.Version{SemVer:"v2.10.0", GitCommit:"9ad53aac42165a5fadc6c87be0dea6b115f93090", GitTreeState:"clean"}
Server: &version.Version{SemVer:"v2.10.0", GitCommit:"9ad53aac42165a5fadc6c87be0dea6b115f93090", GitTreeState:"clean"}
```